**Interactive Analysis and Predictive Modeling of Cardiometabolic Risk Using NHANES 2021–2023**



***1.1 Introduction***


Addressing the growing burden of cardiometabolic disease requires an understanding of the intricate interactions among eating patterns, molecular indicators, and body composition. In order to create a single, comprehensive dataset, we included laboratory results (total cholesterol), anthropometric data (BMI, waist and hip circumferences), and extensive 24-hour dietary recall metrics from NHANES. We investigated the co-occurrence of demographic characteristics, nutritional imbalances, and central adiposity using a suite of interactive Bokeh visualizations, such as stacked macronutrient bar charts, a participant-level dashboard, and scatterplots of BMI vs. age and WHR vs. BMI.

Expanding upon these descriptive studies, we further created and contrasted two prediction models, a random forest classifier and a penalized logistic regression, to differentiate between participants who were normal weight and those who were overweight or obese based on important characteristics. This blend of supervised learning and exploratory visualization provides useful, personalized risk prediction as well as high-level insights into population trends. In order to guarantee that our findings may guide focused interventions without jeopardizing participant confidentially, we consistently placed a high priority on ethical data processing, stressing anonymization, permission, and privacy protections.

***1.2 Now we will go through the codes and results of the analysis***

In [ ]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
df=pd.read_sas(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\DEMO_L.xpt",format="xport")
output_path=r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\Demographic.csv"
df.to_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\Demographic.csv",index=False)


In [ ]:
df=pd.read_sas(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\BMX_L.xpt",format="xport")
output_path2=r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\Body_Measure.csv"
df.to_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\Body_Measure.csv",index=False)

In [ ]:
df=pd.read_sas(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\DR1TOT_L.xpt",format="xport")
output_path3=r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\Dietary1.csv"
df.to_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\Dietary1.csv",index=False)

In [ ]:
df=pd.read_sas(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\PAQ_L.xpt",format="xport")
output_path4=r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\PhyActivity.csv"
df.to_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\PhyActivity.csv",index=False)

In [ ]:
df=pd.read_sas(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\TCHOL_L.xpt",format="xport")
output_path5=r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\TChol.csv"
df.to_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\TChol.csv",index=False)

In [ ]:
import os


In [ ]:
paths = {
    'demo':   r"C:/Users/User/OneDrive/Desktop/T1- 2025/SIT731-Data Wrangling/1.7HD/Demographic.csv",
    'body':  r"C:/Users/User/OneDrive/Desktop/T1- 2025/SIT731-Data Wrangling/1.7HD/Body_Measure.csv",
    'diet': r"C:/Users/User/OneDrive/Desktop/T1- 2025/SIT731-Data Wrangling/1.7HD/Dietary1.csv",
    'activity': r"C:/Users/User/OneDrive/Desktop/T1- 2025/SIT731-Data Wrangling/1.7HD/PhyActivity.csv",
    'chol':    r"C:/Users/User/OneDrive/Desktop/T1- 2025/SIT731-Data Wrangling/1.7HD/TChol.csv"
}

# 2. Read SEQNs from each file into a set and list
seqn_sets = {}
seqn_lists = {}
for name, path in paths.items():
    df = pd.read_csv(path, usecols=['SEQN'])
    seqn_sets[name] = set(df['SEQN'])
    seqn_lists[name] = df['SEQN'].tolist()

# 3. Find the intersection of all SEQNs
common_seqns = set.intersection(*seqn_sets.values())

# 4. Build a summary report
report = []
for name in paths:
    total = len(seqn_lists[name])
    common = len(common_seqns)
    unique = total - common
    report.append({
        'Dataset': name,
        'Total SEQNs': total,
        'Common SEQNs': common,
        'Unique SEQNs': unique
    })
report_df = pd.DataFrame(report)

# 5. Check row‐order consistency against Demographics
demo_order = seqn_lists['demo']
order_report = []
for name in paths:
    same_order = (seqn_lists[name] == demo_order)
    order_report.append({
        'Dataset': name,
        'Same Order as Demographics?': same_order
    })
order_df = pd.DataFrame(order_report)

# 6. Print results
print("=== SEQN Comparison ===")
print(report_df.to_string(index=False))
print("\n=== Order Consistency ===")
print(order_df.to_string(index=False))

=== SEQN Comparison ===
 Dataset  Total SEQNs  Common SEQNs  Unique SEQNs
    demo        11933          6337          5596
    body         8860          6337          2523
    diet         8860          6337          2523
activity         8153          6337          1816
    chol         8068          6337          1731

=== Order Consistency ===
 Dataset  Same Order as Demographics?
    demo                         True
    body                        False
    diet                        False
activity                        False
    chol                        False


So here we see that the data isn't consistent through all the 5 datasets. Hence we join them using Left Join to maintain maximum data.

In [ ]:
#Loading into Dataframes
demo_df     = pd.read_csv(paths["demo"])
body_df     = pd.read_csv(paths["body"])
diet_df     = pd.read_csv(paths["diet"])
activity_df = pd.read_csv(paths["activity"])
chol_df     = pd.read_csv(paths["chol"])

#Standardizing SEQN dtype
for df in (demo_df, body_df, diet_df, activity_df, chol_df):
    df["SEQN"] = df["SEQN"].astype(int)

In [ ]:
#Iterative left join
master = demo_df.copy()
master = master.merge(body_df,     on="SEQN", how="left")
master = master.merge(diet_df,     on="SEQN", how="left")
master = master.merge(activity_df, on="SEQN", how="left")
master = master.merge(chol_df,     on="SEQN", how="left")

#Checking if join was successful
print("Rows,Cols in master:", master.shape)
print(master.head())

Rows,Cols in master: (11933, 225)
     SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  130378      12.0       2.0       1.0      43.0       NaN       5.0   
1  130379      12.0       2.0       1.0      66.0       NaN       3.0   
2  130380      12.0       2.0       2.0      44.0       NaN       2.0   
3  130381      12.0       2.0       2.0       5.0       NaN       5.0   
4  130382      12.0       2.0       1.0       2.0       NaN       3.0   

   RIDRETH3  RIDEXMON  RIDEXAGM  ...  PAD790Q  PAD790U  PAD800       PAD810Q  \
0       6.0       2.0       NaN  ...      3.0     b'W'    45.0  3.000000e+00   
1       3.0       2.0       NaN  ...      4.0     b'W'    45.0  3.000000e+00   
2       2.0       1.0       NaN  ...      1.0     b'W'    20.0  5.397605e-79   
3       7.0       1.0      71.0  ...      NaN      NaN     NaN           NaN   
4       3.0       2.0      34.0  ...      NaN      NaN     NaN           NaN   

   PAD810U  PAD820  PAD680       WTPH2YR  LBXT

In [ ]:
#Saving the merged file
base_dir=r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD"
out_path = os.path.join(base_dir, "NHANES_2021_23_master.csv")
master.to_csv(out_path, index=False)
print("Master CSV written to:", out_path)

Master CSV written to: C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_2021_23_master.csv


In [ ]:
master_path = r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_2021_23_master.csv"
df = pd.read_csv(master_path)
#Printing as a list
print(df.columns.tolist())

['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMDBORN4', 'DMDYRUSR', 'DMDEDUC2', 'DMDMARTZ', 'RIDEXPRG', 'DMDHHSIZ', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ', 'DMDHRMAZ', 'DMDHSEDZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVSTRA', 'SDMVPSU', 'INDFMPIR', 'BMDSTATS', 'BMXWT', 'BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHEAD', 'BMIHEAD', 'BMXHT', 'BMIHT', 'BMXBMI', 'BMDBMIC', 'BMXLEG', 'BMILEG', 'BMXARML', 'BMIARML', 'BMXARMC', 'BMIARMC', 'BMXWAIST', 'BMIWAIST', 'BMXHIP', 'BMIHIP', 'WTDRD1', 'WTDR2D', 'DR1DRSTZ', 'DR1EXMER', 'DRABF', 'DRDINT', 'DR1DBIH', 'DR1DAY', 'DR1LANG', 'DR1MRESP', 'DR1HELP', 'DBQ095Z', 'DBD100', 'DRQSPREP', 'DR1STY', 'DR1SKY', 'DRQSDIET', 'DRQSDT1', 'DRQSDT2', 'DRQSDT3', 'DRQSDT4', 'DRQSDT5', 'DRQSDT6', 'DRQSDT7', 'DRQSDT8', 'DRQSDT9', 'DRQSDT10', 'DRQSDT11', 'DRQSDT12', 'DRQSDT91', 'DR1TNUMF', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR', 'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCH

In [ ]:
#Loading the merged dataset
master_path = r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_2021_23_master.csv"
df = pd.read_csv(master_path)

#Computing missing values counts and percentage
missing = df.isna().sum().to_frame('missing_count')
missing['missing_pct'] = missing['missing_count'] / len(df) * 100

#Sorting and displaying the top 10 by missing percentage
missing = missing.sort_values('missing_pct', ascending=False)
print("Top 10 columns by missingness:\n")
print(missing.head(10))

Top 10 columns by missingness:

          missing_count  missing_pct
BMIHEAD           11933   100.000000
DRQSDT5           11931    99.983240
DRD370PQ          11929    99.966480
DRQSDT6           11926    99.941339
DRD370JQ          11921    99.899439
DRQSDT12          11920    99.891058
DRD350JQ          11918    99.874298
DRD370LQ          11915    99.849158
BMIRECUM          11915    99.849158
DRQSDT8           11906    99.773737


In [ ]:
print("=== DataFrame Info ===")
df.info()
print("\n=== First 5 Rows ===")
print(df.head())
print("\n=== Descriptive Statistics ===")
print(df.describe())

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Columns: 225 entries, SEQN to LBDTCSI
dtypes: float64(222), int64(1), object(2)
memory usage: 20.5+ MB

=== First 5 Rows ===
     SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  130378      12.0       2.0       1.0      43.0       NaN       5.0   
1  130379      12.0       2.0       1.0      66.0       NaN       3.0   
2  130380      12.0       2.0       2.0      44.0       NaN       2.0   
3  130381      12.0       2.0       2.0       5.0       NaN       5.0   
4  130382      12.0       2.0       1.0       2.0       NaN       3.0   

   RIDRETH3  RIDEXMON  RIDEXAGM  ...  PAD790Q  PAD790U  PAD800       PAD810Q  \
0       6.0       2.0       NaN  ...      3.0     b'W'    45.0  3.000000e+00   
1       3.0       2.0       NaN  ...      4.0     b'W'    45.0  3.000000e+00   
2       2.0       1.0       NaN  ...      1.0     b'W'    20.0  5.397605e-79   
3       7.0      

Now, we will be replacing SAS‐style sentinel values (extremely small floats <1×10⁻⁸ and large flag codes ≥9000) with NaNs, converting all string/object fields to the categorical dtype, and auto‐casting any numeric columns with fewer than 20 unique values as categories.

In [ ]:
import numpy as np

In [ ]:
# Only clean numeric columns other than SEQN
num_cols = (
    df
    .select_dtypes(include=['float64','int64'])
    .columns
    .difference(['SEQN'])
)

for c in num_cols:
    df[c] = (
        df[c]
        .mask(df[c].abs() < 1e-8, np.nan)
        .mask(df[c] >= 9000, np.nan)
    )

# Clean byte-strings in object columns
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category')

# auto‐cast low‐cardinality numeric columns to category
for col in df.select_dtypes(include=['float64','int64']).columns:
    if df[col].nunique(dropna=False) < 20:   # tweak 20 up or down
        df[col] = df[col].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Columns: 225 entries, SEQN to LBDTCSI
dtypes: category(126), float64(98), int64(1)
memory usage: 10.5 MB


Running that will give us the complete set of variables now treated as categories—so we can scan and confirm that only the true code/flag fields (and not continuous measures like nutrient intakes) ended up in this list.

In [ ]:
# Get all category columns
cat_cols = df.select_dtypes(include='category').columns.tolist()


print(f"Total categorical columns: {len(cat_cols)}")
for col in cat_cols:
    print(col)


Total categorical columns: 126
SDDSRVYR
RIDSTATR
RIAGENDR
RIDRETH1
RIDRETH3
RIDEXMON
DMQMILIZ
DMDBORN4
DMDYRUSR
DMDEDUC2
DMDMARTZ
RIDEXPRG
DMDHHSIZ
DMDHRGND
DMDHRAGZ
DMDHREDZ
DMDHRMAZ
DMDHSEDZ
SDMVSTRA
SDMVPSU
BMDSTATS
BMIWT
BMIRECUM
BMIHEAD
BMIHT
BMDBMIC
BMILEG
BMIARML
BMIARMC
BMIWAIST
BMIHIP
DR1DRSTZ
DR1EXMER
DRABF
DRDINT
DR1DAY
DR1LANG
DR1MRESP
DR1HELP
DBQ095Z
DBD100
DRQSPREP
DR1STY
DR1SKY
DRQSDIET
DRQSDT1
DRQSDT2
DRQSDT3
DRQSDT4
DRQSDT5
DRQSDT6
DRQSDT7
DRQSDT8
DRQSDT9
DRQSDT10
DRQSDT11
DRQSDT12
DRQSDT91
DR1_300
DR1TWSZ
DRD340
DRD350A
DRD350AQ
DRD350B
DRD350BQ
DRD350C
DRD350CQ
DRD350D
DRD350DQ
DRD350E
DRD350EQ
DRD350F
DRD350FQ
DRD350G
DRD350GQ
DRD350H
DRD350HQ
DRD350I
DRD350IQ
DRD350J
DRD350JQ
DRD350K
DRD360
DRD370A
DRD370AQ
DRD370B
DRD370C
DRD370CQ
DRD370D
DRD370DQ
DRD370E
DRD370EQ
DRD370F
DRD370FQ
DRD370G
DRD370GQ
DRD370H
DRD370HQ
DRD370I
DRD370IQ
DRD370J
DRD370JQ
DRD370K
DRD370KQ
DRD370L
DRD370LQ
DRD370M
DRD370MQ
DRD370N
DRD370NQ
DRD370O
DRD370OQ
DRD370P
DRD370PQ
DRD370Q
DRD370QQ

Now we will deal with the duplicate entries in the dataset.

In [ ]:
# Computing frequency of each SEQN
seqn_counts = df['SEQN'].value_counts()

# Filtering to those SEQNs that appear more than once
dup_seqns = seqn_counts[seqn_counts > 1]

print(f"Number of SEQNs repeated more than once: {dup_seqns.size}")
print("\nSample of duplicated SEQNs and their counts:")
print(dup_seqns.head())

dup_exactly_two = seqn_counts[seqn_counts == 2]
print(f"\nNumber of SEQNs repeated exactly twice: {dup_exactly_two.size}")
print(dup_exactly_two.head())


Number of SEQNs repeated more than once: 0

Sample of duplicated SEQNs and their counts:
Series([], Name: count, dtype: int64)

Number of SEQNs repeated exactly twice: 0
Series([], Name: count, dtype: int64)


Since there are no SEQNs repeated, we can skip any de-duplication step—the dataset already has exactly one row per person.

In [ ]:
miss = df.isna().sum().to_frame(name='missing_count')
miss['missing_pct'] = miss['missing_count'] / len(df) * 100

miss = miss.sort_values('missing_pct', ascending=False)

print("Top 20 most‐missing columns:\n", miss.head(20))
print("\nBottom 20 least‐missing columns:\n", miss.tail(20))

Top 20 most‐missing columns:
           missing_count  missing_pct
BMIHEAD           11933   100.000000
DRQSDT5           11931    99.983240
DRD370PQ          11929    99.966480
DRQSDT6           11926    99.941339
DRD370JQ          11921    99.899439
DRQSDT12          11920    99.891058
DRD350JQ          11918    99.874298
BMIRECUM          11915    99.849158
DRD370LQ          11915    99.849158
WTDR2D            11908    99.790497
DRQSDT8           11906    99.773737
DRQSDT11          11901    99.731836
DRQSDT10          11900    99.723456
DRQSDT4           11889    99.631275
DRD370QQ          11888    99.622894
WTDRD1            11886    99.606134
WTPH2YR           11884    99.589374
DRD370CQ          11873    99.497193
DRD350CQ          11871    99.480432
BMXHEAD           11863    99.413391

Bottom 20 least‐missing columns:
           missing_count  missing_pct
BMXBMI             3462    29.011984
BMXHT              3434    28.777340
BMXARMC            3371    28.249392
BMXARML   

In [ ]:
import os

In [ ]:
#Defining the final list of columns to keep
keep_cols = [
    # ID
    'SEQN',
    # Demographics
    'RIAGENDR', 'RIDAGEYR', 'RIDRETH3',
    'DMDBORN4', 'DMDEDUC2', 'DMDMARTZ', 'DMDHHSIZ',
    'WTINT2YR', 'WTMEC2YR', 'INDFMPIR',
    # Body measures
    'BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST', 'BMXHIP',
    # Dietary recall
    'WTDRD1', 'WTDR2D', 'DR1TNUMF',
    'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR', 'DR1TFIBE', 'DR1TTFAT',
    'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
    # Physical activity
    'PAD790Q', 'PAD800', 'PAD810Q', 'PAD820', 'PAD680',
    # Laboratory
    'WTPH2YR', 'LBXTC'
]

#Subset to only the selected columns
df_selected = df[keep_cols].copy()
print("Subset head:\n", df_selected['SEQN'].head())

print("Original shape:", df.shape)
print("Subset shape :", df_selected.shape)
print("Columns kept   :", df_selected.columns.tolist())


Subset head:
 0    130378
1    130379
2    130380
3    130381
4    130382
Name: SEQN, dtype: int64
Original shape: (11933, 225)
Subset shape : (11933, 36)
Columns kept   : ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDBORN4', 'DMDEDUC2', 'DMDMARTZ', 'DMDHHSIZ', 'WTINT2YR', 'WTMEC2YR', 'INDFMPIR', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST', 'BMXHIP', 'WTDRD1', 'WTDR2D', 'DR1TNUMF', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR', 'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL', 'PAD790Q', 'PAD800', 'PAD810Q', 'PAD820', 'PAD680', 'WTPH2YR', 'LBXTC']


In [ ]:
#Saving the pared-down dataset
out_path = os.path.join(base_dir, "NHANES_master_selected.csv")
df_selected.to_csv(out_path, index=False)
print("Selected dataset saved to:", out_path)

Selected dataset saved to: C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_master_selected.csv


Following our domain goals and decisions about variable importance (whether to "keep" or "drop"), we created a definitive list of 36 crucial variables. Total dietary intakes (energy, macronutrients, fiber, cholesterol), summary physical activity metrics (frequency and duration of moderate/vigorous activity plus sedentary time), core demographics (age, gender, race/ethnicity, education, and income), basic anthropometrics (weight, height, BMI, waist and hip circumferences), participant weights for survey-weighted analyses, and the primary laboratory outcome (total serum cholesterol) were among these. Every other column, including metadata flags, highly specialized sub-measures, and micronutrient breakdowns with a lot of missing data, was eliminated because it was thought to be unrelated to our main study concerns.

Now we will deal with the missing values.

In [ ]:
#Separating numeric vs. categorical columns
num_cols = df.select_dtypes(include=['float64','int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()

#Imputing numeric columns with the median
for col in cat_cols:
    # fill missing with “Unknown”
    filled = df[col].astype('object').fillna('Unknown')
    # infer the best dtype and then cast to category
    df[col] = filled.infer_objects().astype('category')

#Imputing categorical columns with "Unknown"
for col in cat_cols:
    df[col] = df[col].astype('object').fillna('Unknown').astype('category')


In [ ]:
#Verifying no missing values remain
missing_after = df.isna().sum().sum()
print(f"Total missing values after imputation: {missing_after}")


Total missing values after imputation: 605353


In [ ]:
#Saving the imputed dataset
out_path = os.path.join(base_dir, "NHANES_master_imputed.csv")
df.to_csv(out_path, index=False)
print(f"Imputed dataset saved to: {out_path}")

Imputed dataset saved to: C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_master_imputed.csv


Now that we’ve imputed all missing values, the next logical step is to detect and tame any extreme outliers in our continuous measures so they don’t dominate our visuals or skew analyses.

In [ ]:
#Load the imputed dataset
base_dir = r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD"
imp_path = os.path.join(base_dir, "NHANES_master_imputed.csv")
df = pd.read_csv(imp_path)

In [ ]:
#Identifying continuous numeric columns (excluding ID)
num_cols = df.select_dtypes(include=['float64','int64']).columns.tolist()
num_cols = [c for c in num_cols if c != 'SEQN']

In [ ]:
#Winsorizing each at the 1st and 99th percentiles
for c in num_cols:
    low, high = df[c].quantile([0.01, 0.99])
    df[c] = df[c].clip(lower=low, upper=high)

#Computing descriptive stats including the 1st and 99th percentiles
winsor_stats = df[num_cols].describe(percentiles=[.01, .25, .5, .75, .99]).T

#Selecting the key columns to inspect
winsor_stats = winsor_stats[['min', '1%', '25%', '50%', '75%', '99%', 'max', 'mean', 'std']]

#Printing the table
print("Winsorized summary (min,1%,25%,50%,75%,99%,max):\n")
print(winsor_stats)


Winsorized summary (min,1%,25%,50%,75%,99%,max):

                  min           1%          25%          50%          75%  \
SDDSRVYR    12.000000    12.000000    12.000000    12.000000    12.000000   
RIDSTATR     1.000000     1.000000     1.000000     2.000000     2.000000   
RIAGENDR     1.000000     1.000000     1.000000     2.000000     2.000000   
RIDAGEYR     1.000000     1.000000    14.000000    38.000000    62.000000   
RIDAGEMN     1.000000     1.000000     7.000000    12.000000    18.000000   
...               ...          ...          ...          ...          ...   
PAD820       5.000000     5.000000    30.000000    45.000000    60.000000   
PAD680      30.000000    30.000000   180.000000   300.000000   480.000000   
WTPH2YR   5266.131584  5720.772538  7134.269094  7966.538487  8484.029655   
LBXTC      101.890000   101.987900   151.000000   178.000000   207.000000   
LBDTCSI      2.636700     2.639637     3.900000     4.600000     5.350000   

                  99%    

In [ ]:
import numpy as np

In [ ]:
# 1. Compute original skews
orig_skews = df[num_cols].skew().sort_values(ascending=False)

# 2. Pick columns with |skew| > 1 to transform
to_log = orig_skews[orig_skews.abs() > 1].index.tolist()

# 3. Create the log1p versions in a new DataFrame
log_df = pd.DataFrame({
    col: np.log1p(df[col])
    for col in to_log
})

# 4. Compute skewness of each log‐column directly from log_df
log_skews = log_df.skew().sort_values(ascending=False)

# 5. Build comparison table, aligning by column name
skew_compare = pd.DataFrame({
    'original_skew': orig_skews[to_log],
    'log1p_skew':    log_skews[to_log]
})

# 6. (Optionally) if you want to keep the log columns in df:
df = pd.concat([df, log_df.add_suffix('_log')], axis=1)

# 7. Show results
print(skew_compare)


          original_skew  log1p_skew
DR1TP225       5.955781    5.573126
DR1TP205       5.945671    5.419029
DR1TP226       5.075711    4.440625
DR1TP184       3.738538    3.637175
DR1TB12A       3.419295    1.619609
...                 ...         ...
DR1TPHOS       1.064115   -0.519477
RIDRETH3       1.041638   -0.064933
RIDSTATR      -1.109200   -1.109200
BMXHT         -1.478349   -1.895137
BMXARML       -1.488429   -2.094417

[72 rows x 2 columns]


After winsorizing at the 1st and 99th percentiles and applying a log₁₊ₓ transformation to our most heavily skewed variables, the vast majority of distributions now lie well within a visually interpretable range. While a small number of series still exhibit post‐log skew values exceeding ±2, this degree of asymmetry will not dominate our interactive Bokeh plots—zooming, panning, and axis scaling will allow users to explore both central tendencies and the tails without a few extreme observations compressing the bulk of the data into a narrow band. More aggressive techniques (e.g. Box–Cox transforms or tighter percentile capping) could further normalize these outliers, but at the cost of interpretability and the risk of obscuring legitimately extreme but valid responses. For our purposes—providing clear, intuitive, and data‐driven visualizations—the combination of winsorizing plus log₁₊ₓ strikes the optimal balance between outlier mitigation and preservation of real variation.

In [ ]:
import pandas as pd
import numpy as np
import io
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure
from bokeh.models import (ColumnDataSource, HoverTool, CategoricalColorMapper,
                         Legend, LegendItem, NumeralTickFormatter, BoxSelectTool,
                         RangeTool, Slider, CustomJS, RadioButtonGroup, Select,
                         Tabs, Panel, DataTable, TableColumn, Button)
from bokeh.layouts import column, row, gridplot
from bokeh.palettes import Category10, Spectral6, Viridis256
from bokeh.transform import factor_cmap, jitter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the CSV data
data = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_master_imputed.csv')

# VISUALIZATION 1: BMI Distribution by Age and Gender with Hover Info
def create_bmi_scatter():
    # Filter rows with valid BMI and age data
    bmi_data = data[['RIDAGEYR', 'RIAGENDR', 'BMXBMI', 'BMXWT', 'BMXHT', 'BMXWAIST']].dropna(subset=['BMXBMI', 'RIDAGEYR'])

    # Map gender codes to labels
    bmi_data['Gender'] = bmi_data['RIAGENDR'].map({1: 'Male', 2: 'Female'})

    # Create BMI category
    def bmi_category(bmi):
        if bmi < 18.5:
            return 'Underweight'
        elif bmi < 25:
            return 'Normal'
        elif bmi < 30:
            return 'Overweight'
        else:
            return 'Obese'

    bmi_data['BMI_Category'] = bmi_data['BMXBMI'].apply(bmi_category)

    # Prepare data source
    source = ColumnDataSource(data=dict(
        age=bmi_data['RIDAGEYR'],
        bmi=bmi_data['BMXBMI'],
        gender=bmi_data['Gender'],
        weight=bmi_data['BMXWT'],
        height=bmi_data['BMXHT'],
        waist=bmi_data['BMXWAIST'],
        category=bmi_data['BMI_Category']
    ))

    # Create color mappers
    gender_mapper = CategoricalColorMapper(factors=['Male', 'Female'], palette=['#1f77b4', '#ff7f0e'])
    category_mapper = CategoricalColorMapper(
        factors=['Underweight', 'Normal', 'Overweight', 'Obese'],
        palette=['#2ca02c', '#1f77b4', '#ff7f0e', '#d62728']
    )

    # Create the figure
    p = figure(
        title='BMI Distribution by Age and Gender',
        x_axis_label='Age (years)',
        y_axis_label='BMI (kg/m²)',
        width=800,
        height=500,
        tools='pan,box_select,zoom_in,zoom_out,reset,save'
    )

    # Add scatter points with both gender and category information
    gender_scatter = p.circle(
        'age', 'bmi',
        source=source,
        size=12,
        alpha=0.7,
        color={'field': 'gender', 'transform': gender_mapper},
        line_color='black'
    )

    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ('Age', '@age years'),
            ('Gender', '@gender'),
            ('BMI', '@bmi{0.0} kg/m²'),
            ('Category', '@category'),
            ('Weight', '@weight{0.0} kg'),
            ('Height', '@height{0.0} cm'),
            ('Waist', '@waist{0.0} cm')
        ],
        renderers=[gender_scatter]
    )
    p.add_tools(hover)

    # Add BMI category reference lines
    p.line([0, 100], [18.5, 18.5], line_color='green', line_dash='dashed', line_width=2, legend_label='Underweight < 18.5')
    p.line([0, 100], [25, 25], line_color='blue', line_dash='dashed', line_width=2, legend_label='Normal 18.5-24.9')
    p.line([0, 100], [30, 30], line_color='orange', line_dash='dashed', line_width=2, legend_label='Overweight 25-29.9')

    # Customize legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"

    return p

***Visualization 1: BMI Distribution by Age and Gender***

Each participant's BMI is plotted against their age in this scatter plot, with dots colored according to gender (orange for females, blue for males). Standard BMI standards for underweight (< 18.5), normal (18.5–24.9), and overweight (25–29.9) are shown by dashed horizontal lines.

***Key Insights***
- Predominance of overweight and obesity: Most participants fall above the normal BMI range, with a substantial cluster in the obese category (BMI ≥ 30) and very few underweight observations.
- Variations in distribution by gender: Female BMIs (orange) range from roughly 15 to 75 kg/m², but male BMIs (blue) are more closely clustered around 20 to 45 kg/m².
- Extreme numbers for a range of ages: All age groups are affected by extreme obesity, as evidenced by the fact that the two highest BMIs (over 60 kg/m2) are seen in both younger and older persons.
- Association between central adiposity: The relationship between central fat distribution and metabolic risk is shown by interactive hover, which shows that those with higher BMIs frequently have larger waist measurements.

***Implications***
- The high rate of overweight/obesity in this population suggests a higher risk of chronic diseases.
- Females' greater BMI variability raises the possibility that gender-specific therapies are required.
- By displaying age, gender, BMI, category, weight, height, and waist, the interactive hover functionality allows for comprehensive, individual-level assessment and focused preventative measures.

In [ ]:
# VISUALIZATION 2: Interactive Macronutrient Breakdown by Participant
def create_macronutrient_chart():
    # Filter to participants with dietary data
    diet_data = data[['SEQN', 'RIDAGEYR', 'RIAGENDR', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TTFAT']].dropna()

    # Calculate macronutrient percentages
    diet_data['Protein_pct'] = diet_data['DR1TPROT'] * 4 / diet_data['DR1TKCAL'] * 100
    diet_data['Carbs_pct'] = diet_data['DR1TCARB'] * 4 / diet_data['DR1TKCAL'] * 100
    diet_data['Fat_pct'] = diet_data['DR1TTFAT'] * 9 / diet_data['DR1TKCAL'] * 100

    # Map gender codes to labels
    diet_data['Gender'] = diet_data['RIAGENDR'].map({1: 'Male', 2: 'Female'})

    # Create age category
    def age_category(age):
        if age < 18:
            return 'Child/Teen (<18)'
        elif age < 35:
            return 'Young Adult (18-34)'
        elif age < 50:
            return 'Middle Age (35-49)'
        elif age < 65:
            return 'Older Adult (50-64)'
        else:
            return 'Senior (65+)'

    diet_data['Age_Group'] = diet_data['RIDAGEYR'].apply(age_category)

    # Create sorted indices by total calories
    sorted_indices = diet_data.sort_values('DR1TKCAL').index

    # Add participant ID for display using SEQN
    diet_data['Participant'] = diet_data['SEQN'].astype(str)

    # Prepare data
    participants = diet_data['Participant'].tolist()
    protein = diet_data['Protein_pct'].round(1).tolist()
    carbs = diet_data['Carbs_pct'].round(1).tolist()
    fat = diet_data['Fat_pct'].round(1).tolist()
    calories = diet_data['DR1TKCAL'].round(0).astype(int).tolist()
    gender = diet_data['Gender'].tolist()
    age = diet_data['RIDAGEYR'].tolist()
    age_group = diet_data['Age_Group'].tolist()

    # Create data for stacked bars
    protein_vals = np.array(protein)
    carbs_vals = np.array(carbs)
    fat_vals = np.array(fat)

    # Calculate cumulative values for stacking
    protein_right = protein_vals
    carbs_right = protein_vals + carbs_vals
    fat_right = protein_vals + carbs_vals + fat_vals

    # Create data source
    source = ColumnDataSource(data=dict(
        participants=participants,
        protein=protein_vals,
        carbs=carbs_vals,
        fat=fat_vals,
        protein_right=protein_right,
        carbs_right=carbs_right,
        fat_right=fat_right,
        calories=calories,
        gender=gender,
        age=age,
        age_group=age_group,
        y_range=list(range(len(participants)))
    ))

    # Create figure
    p = figure(
    y_range=participants,
    title="Macronutrient Distribution by Participant",
    width=800, height=400,
    x_range=(0, 100),
    tools="pan,box_zoom,reset,save")

    # Add horizontal bars for each macronutrient
    protein_bar = p.hbar(
    y='y_range', right='protein_right', height=0.8,
    source=source, color="#1f77b4", legend_label="Protein")

    # For carbs
    carbs_bar = p.hbar(
    y='y_range', right='carbs_right', left='protein_right',
    height=0.8, source=source, color="#ff7f0e",
    legend_label="Carbohydrates")

    # For fat
    fat_bar = p.hbar(
    y='y_range', right='fat_right', left='carbs_right',
    height=0.8, source=source, color="#2ca02c",
    legend_label="Fat")

    # Configure hover
    custom_hover = HoverTool(
    renderers=[protein_bar, carbs_bar, fat_bar],
    mode='mouse',
    tooltips=[
        ("Participant", "@participants"),
        ("Age",         "@age years (@age_group)"),
        ("Gender",      "@gender"),
        ("Calories",    "@calories kcal"),
        ("Protein",     "@protein%"),
        ("Carbs",       "@carbs%"),
        ("Fat",         "@fat%")
    ])

    p.add_tools(custom_hover)

    # Customize figure
    p.xaxis.axis_label = "Percentage of Total Calories (%)"
    p.yaxis.axis_label = "Participant"
    p.legend.location = "bottom_right"
    p.legend.orientation = "horizontal"
    p.legend.click_policy = "hide"

    # Add reference lines for recommended macronutrient ranges
    p.add_layout(Legend(items=[
        LegendItem(label="Recommended Protein: 10-35%", renderers=[
            p.line([10, 10], [0, len(participants)], line_color="blue", line_dash="dashed"),
            p.line([35, 35], [0, len(participants)], line_color="blue", line_dash="dashed")
        ]),
        LegendItem(label="Recommended Carbs: 45-65%", renderers=[
            p.line([45, 45], [0, len(participants)], line_color="orange", line_dash="dashed"),
            p.line([65, 65], [0, len(participants)], line_color="orange", line_dash="dashed")
        ]),
        LegendItem(label="Recommended Fat: 20-35%", renderers=[
            p.line([20, 20], [0, len(participants)], line_color="green", line_dash="dashed"),
            p.line([35, 35], [0, len(participants)], line_color="green", line_dash="dashed")
        ])
    ]))

    return p

***Visualization 2: Macronutrient Distribution by Participant***

This stacked horizontal bar chart shows, for each participant, the percentage of total calories obtained from protein (blue), carbohydrates (orange), and fat (green). Vertical dashed lines mark the recommended ranges: protein 10–35% (blue), carbs 45–65% (orange), and fat 20–35% (green). Hovering over any bar segment reveals the participant’s age (and age group), gender, total calories, and exact macronutrient percentages.

***Key Insights***
- Deficiencies in protein: Protein contributions range from 7.2% to 21.9%, with many individuals falling short of the recommended minimum of 10%.
- Extremes of carbs: Only those with a carbohydrate proportion of 45% or more meet the recommended range of 23.8% to 63.6%. A 2-year-old boy notably reaches 63.6% carbohydrates.
- Excess fat: There is a range of 22.1% to 54.8% fat intake; a 68-year-old woman's diet is particularly high at 54.8%, which is significantly more than the 35% top limit.
- Few people are completely compliant: The majority of diets in this area are unbalanced, as only a tiny percentage of people have all three macronutrients completely within the advised ranges.

***Implications***
- Dietary patterns are highly variable, with many participants failing to meet one or more macronutrient recommendations.
- Widespread low protein intake (< 10% of calories) suggests a need to increase protein sources in many diets.
- In certain people, abnormally high percentages of carbohydrates (such as 63.6% in a 2-year-old) indicate an imbalance in their energy sources.
- Moderation is advised since high fat intake (up to 54.8% in a 68-year-old) may increase cardiometabolic risk.
- A small percentage of participants fall into each of the three suggested categories, underscoring the wide range of potential for individualized dietary advice.

In [ ]:
# VISUALIZATION 3: Waist-to-Hip Ratio vs. BMI with Dietary Information
def create_waist_hip_bmi_plot():
    # Filter data with complete body measurements
    whr_data = data[['RIDAGEYR', 'RIAGENDR', 'BMXBMI', 'BMXWAIST', 'BMXHIP',
                    'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TTFAT']].copy()

    # Drop rows with missing waist or hip measurements
    whr_data = whr_data.dropna(subset=['BMXWAIST', 'BMXHIP', 'BMXBMI'])

    # Calculate waist-to-hip ratio
    whr_data['WHR'] = whr_data['BMXWAIST'] / whr_data['BMXHIP']

    # Map gender codes to labels
    whr_data['Gender'] = whr_data['RIAGENDR'].map({1: 'Male', 2: 'Female'})

    # Create risk categories based on WHO guidelines
    def whr_risk_category(row):
        if row['Gender'] == 'Male':
            if row['WHR'] < 0.9:
                return 'Low Risk'
            elif row['WHR'] < 1.0:
                return 'Moderate Risk'
            else:
                return 'High Risk'
        else:  # Female
            if row['WHR'] < 0.8:
                return 'Low Risk'
            elif row['WHR'] < 0.85:
                return 'Moderate Risk'
            else:
                return 'High Risk'

    whr_data['Risk_Category'] = whr_data.apply(whr_risk_category, axis=1)

    # Create BMI categories
    def bmi_category(bmi):
        if bmi < 18.5:
            return 'Underweight'
        elif bmi < 25:
            return 'Normal'
        elif bmi < 30:
            return 'Overweight'
        else:
            return 'Obese'

    whr_data['BMI_Category'] = whr_data['BMXBMI'].apply(bmi_category)

    # Calculate diet quality if data is available
    whr_data['Has_Diet_Data'] = ~whr_data['DR1TKCAL'].isna()
    whr_data.loc[whr_data['Has_Diet_Data'], 'Calories'] = whr_data.loc[whr_data['Has_Diet_Data'], 'DR1TKCAL']

    # Create a source for the plot
    source = ColumnDataSource(data=dict(
        bmi=whr_data['BMXBMI'],
        whr=whr_data['WHR'],
        waist=whr_data['BMXWAIST'],
        hip=whr_data['BMXHIP'],
        gender=whr_data['Gender'],
        risk=whr_data['Risk_Category'],
        bmi_cat=whr_data['BMI_Category'],
        age=whr_data['RIDAGEYR'],
        calories=whr_data['DR1TKCAL'],
        has_diet=whr_data['Has_Diet_Data']
    ))

    # Create color mappers
    risk_mapper = CategoricalColorMapper(
        factors=['Low Risk', 'Moderate Risk', 'High Risk'],
        palette=['#2ca02c', '#ff7f0e', '#d62728']
    )

    gender_mapper = CategoricalColorMapper(
        factors=['Male', 'Female'],
        palette=['#1f77b4', '#ff7f0e']
    )

    # Create the figure
    p = figure(
        title='Waist-to-Hip Ratio vs. BMI',
        x_axis_label='BMI (kg/m²)',
        y_axis_label='Waist-to-Hip Ratio',
        width=800,
        height=500,
        tools='pan,box_select,zoom_in,zoom_out,reset,save'
    )

    # Add scatter points
    scatter = p.circle(
        'bmi', 'whr',
        source=source,
        size=12,
        fill_color={'field': 'risk', 'transform': risk_mapper},
        line_color={'field': 'gender', 'transform': gender_mapper},
        line_width=2,
        alpha=0.7
    )

    # Add reference lines
    # Male WHR risk threshold
    p.line([15, 50], [0.9, 0.9], line_color='blue', line_dash='dashed', line_width=2, legend_label='Male WHR Risk Threshold')
    # Female WHR risk threshold
    p.line([15, 50], [0.8, 0.8], line_color='orange', line_dash='dashed', line_width=2, legend_label='Female WHR Risk Threshold')
    # BMI reference lines
    p.line([25, 25], [0.6, 1.2], line_color='green', line_dash='dotted', line_width=2, legend_label='BMI = 25 (Overweight)')
    p.line([30, 30], [0.6, 1.2], line_color='red', line_dash='dotted', line_width=2, legend_label='BMI = 30 (Obese)')

    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ('Gender', '@gender'),
            ('Age', '@age years'),
            ('BMI', '@bmi{0.0} kg/m²'),
            ('BMI Category', '@bmi_cat'),
            ('WHR', '@whr{0.00}'),
            ('Risk Category', '@risk'),
            ('Waist', '@waist{0.0} cm'),
            ('Hip', '@hip{0.0} cm'),
            ('Calories', '@calories{0,0} kcal')
        ],
        renderers=[scatter]
    )
    p.add_tools(hover)

    # Customize legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"

    return p

***Visualization 3: Waist-to-Hip Ratio vs. BMI***

Each participant's waist-to-hip ratio (WHR, y-axis) is shown against their BMI (x-axis) in this scatter plot. Points are shown by gender (blue = Male, orange = Female) and WHR risk group (green = Low Risk, orange = Moderate Risk, red = High Risk). WHR criteria are indicated by dashed horizontal lines (0.9 for men and 0.8 for women); BMI = 25 (overweight) and BMI = 30 (obesity) are indicated by dotted vertical lines. Age, BMI, WHR, risk category, body measurements, and daily caloric intake are displayed when hovered over.

***Key Insights***
- Overweight and obese people have high WHRs: Every person who meets WHR risk thresholds has a BMI of at least 25.
- Gender disparities: While females cluster closer to their 0.8 threshold, male markers (blue outlines) frequently appear higher on the WHR axis—many guys exceed 0.9.
- Divergent fat distribution: Some individuals with very high BMI (e.g., 64 kg/m²) remain in a Low Risk WHR category, illustrating different patterns of central adiposity.
- Complementary measurements: By identifying those who have excess abdominal fat, WHR refines risk, whereas BMI alone would classify many as obese.

***Implications***
- Individuals who have a BMI of 30 or above with a WHR above the limits are at a higher risk of cardiometabolic problems and ought to receive priority attention.
- Despite having a high body mass, WHR identifies people with a healthy fat distribution, adding nuance beyond BMI.
- The interactive hover that connects WHR/BMI and caloric intake reveals that people with the highest anthropometric risk tend to consume more calories, which suggests nutritional counseling targets.

In [ ]:
# VISUALIZATION 4: Interactive Nutritional Intake Analysis
def create_nutrient_analysis():
    # Select nutritional columns of interest
    nutrient_cols = [
        'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TFIBE', 'DR1TSUGR',
        'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
        'DR1TSODI', 'DR1TPOTA', 'DR1TCALC', 'DR1TIRON', 'DR1TZINC'
    ]

    # Filter participants with complete nutritional data
    nutrient_data = data[['SEQN', 'RIDAGEYR', 'RIAGENDR'] + nutrient_cols].dropna(subset=['DR1TKCAL'])

    # Map gender codes to labels
    nutrient_data['Gender'] = nutrient_data['RIAGENDR'].map({1: 'Male', 2: 'Female'})

    # Create participant ID
    nutrient_data['Participant_ID'] = nutrient_data['SEQN'].astype(str)

    # Create readable nutrient names mapping
    nutrient_names = {
        'DR1TKCAL': 'Calories (kcal)',
        'DR1TPROT': 'Protein (g)',
        'DR1TCARB': 'Carbohydrates (g)',
        'DR1TFIBE': 'Fiber (g)',
        'DR1TSUGR': 'Sugar (g)',
        'DR1TTFAT': 'Total Fat (g)',
        'DR1TSFAT': 'Saturated Fat (g)',
        'DR1TMFAT': 'Monounsaturated Fat (g)',
        'DR1TPFAT': 'Polyunsaturated Fat (g)',
        'DR1TCHOL': 'Cholesterol (mg)',
        'DR1TSODI': 'Sodium (mg)',
        'DR1TPOTA': 'Potassium (mg)',
        'DR1TCALC': 'Calcium (mg)',
        'DR1TIRON': 'Iron (mg)',
        'DR1TZINC': 'Zinc (mg)'
    }

    # Calculate reference values (simplified - would be more accurate with gender/age specific values)
    reference_values = {
        'DR1TKCAL': 2000,        # Base reference
        'DR1TPROT': 50,          # g/day
        'DR1TCARB': 275,         # g/day
        'DR1TFIBE': 25,          # g/day
        'DR1TSUGR': 50,          # g/day
        'DR1TTFAT': 65,          # g/day
        'DR1TSFAT': 20,          # g/day
        'DR1TMFAT': 25,          # g/day
        'DR1TPFAT': 25,          # g/day
        'DR1TCHOL': 300,         # mg/day
        'DR1TSODI': 2300,        # mg/day
        'DR1TPOTA': 4700,        # mg/day
        'DR1TCALC': 1000,        # mg/day
        'DR1TIRON': 18,          # mg/day
        'DR1TZINC': 11           # mg/day
    }

    # Create source for the initial nutrient (calories)
    initial_nutrient = 'DR1TKCAL'
    source = ColumnDataSource(data=dict(
        participants=nutrient_data['Participant_ID'].tolist(),
        values=nutrient_data[initial_nutrient].tolist(),
        gender=nutrient_data['Gender'].tolist(),
        age=nutrient_data['RIDAGEYR'].tolist()
    ))

    # Create figure
    p = figure(
        y_range=source.data['participants'],
        title=f"Nutrient Intake: {nutrient_names[initial_nutrient]}",
        width=800,
        height=400,
        x_axis_label=nutrient_names[initial_nutrient],
        tools="pan,box_zoom,reset,save"
    )

    # Add horizontal bars
    bars = p.hbar(
        y='participants', right='values', height=0.9, source=source,
        color=factor_cmap('gender', ['#1f77b4','#ff7f0e'], ['Male','Female']),
        line_color='black', line_width=1)

    # Add reference line
    ref_value = reference_values[initial_nutrient]
    ref_line = p.line([ref_value, ref_value],
                      [0, len(nutrient_data)], line_color="red", line_dash="dashed", line_width=2)

    # Add reference line label
    ref_label = p.text(x=[ref_value * 1.05], y=[len(nutrient_data) * 0.9],
                      text=['Reference Value'], text_color="red")

    # Configure hover
    hover = HoverTool(
        renderers=[bars],
        tooltips=[
            ("Participant", "@participants"),
            ("Age",         "@age years"),
            ("Gender",      "@gender"),
            (nutrient_names[initial_nutrient], "@values{0,0.0}")],
        mode='mouse')
    p.add_tools(hover)

    # Customize figure
    p.xaxis.axis_label = nutrient_names[initial_nutrient]
    p.yaxis.axis_label = "Participant"

    # Create nutrient selector
    nutrient_select = Select(
        title="Select Nutrient:",
        value=nutrient_names[initial_nutrient],
        options=list(nutrient_names.values())
    )

    display_to_code = {v: k for k, v in nutrient_names.items()}

    # JavaScript callback to update plot when nutrient is changed
    callback = CustomJS(
        args=dict(
            source=source,
            p=p,
            xaxis=p.xaxis[0],
            ref_line=ref_line,
            ref_label=ref_label,
            nutrient_data=nutrient_data.to_dict('list'),
            reference_values=reference_values,
            hover=hover,
            display_to_code=display_to_code
        ),
        code="""
            // 1) Figure out the selection
            const sel_name = cb_obj.value;
            const sel_code = display_to_code[sel_name];
            const rv     = reference_values[sel_code];

            // 2) Swap in the new bar values
            source.data['values'] = nutrient_data[sel_code];
            source.change.emit();

            // 3) Rebuild the hover-tooltips array completely
            const new_tips = [
              ["Participant", "@participants"],
              ["Age",         "@age years"],
              ["Gender",      "@gender"],
              [ sel_name, "@values{0,0.0}" ]
            ];
            hover.setv({ tooltips: new_tips });

            // 4) Update title and axis label
            p.title.text = "Nutrient Intake: " + sel_name;
            xaxis.setv({ axis_label: sel_name });
            xaxis.change.emit();

            // 5) Move the dashed reference line
            ref_line.setv({ xs: [rv, rv] });

            // 6) Reposition the "Reference Value" text
            const ds = ref_label.data_source;
            ds.data['x'] = [ rv * 1.05 ];
            ds.change.emit();

        """
    )

    nutrient_select.js_on_change('value', callback)


    # Create layout
    layout = column(nutrient_select, p)

    return layout

***Visualization 4: Interactive Nutrient Analysis***

Using the dropdown menu, you may select any nutrient in this horizontal bar chart, including vitamins, minerals, calories, and macronutrients. One participant's intake is represented by each bar. The suggested daily goal for the chosen nutrient is indicated by a red dashed reference line.

***Key Insights***
- Large variety of calories: Energy consumption ranges from 1,200 to 3,850 kcal per day, with the 2,000 kcal recommended at its center.
- Extensive lack of fiber: Most participants don't meet the recommended daily intake of 25 grams of fiber.
- High fluctuation in sugar levels: Consumption of sugar varies from about 38 g to about 228 g, with many people going over the 50 g limit.
- Frequent overconsumption of sodium: Many exceed the recommended daily intake of 2,300 mg of sodium.
- Protein adequacy with outliers. Most meet the 50 g/day protein reference, but intake varies widely (29–132 g).

***Implications***
- Static charts fail to identify nutrient-specific imbalances, but interactive selection does.
- Participants who require attention for sodium, sugar, or fiber are immediately identified by reference lines.
- This tool facilitates the quick detection of outliers and directs nutrient-specific individualized dietary recommendations.

In [ ]:
# VISUALIZATION 5: Health Metrics and Dietary Pattern Dashboard
def create_health_diet_dashboard():
    # Prepare data for dashboard
    dashboard_data = data[['SEQN', 'RIDAGEYR', 'RIAGENDR', 'BMXBMI', 'BMXWT',
                          'BMXHT', 'BMXWAIST', 'BMXHIP', 'LBXTC',
                          'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TFIBE',
                          'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI']].copy()

    # Map gender codes to labels
    dashboard_data['Gender'] = dashboard_data['RIAGENDR'].map({1: 'Male', 2: 'Female'})

    # Calculate waist-to-hip ratio if data is available
    mask = (~dashboard_data['BMXWAIST'].isna()) & (~dashboard_data['BMXHIP'].isna())
    dashboard_data.loc[mask, 'WHR'] = dashboard_data.loc[mask, 'BMXWAIST'] / dashboard_data.loc[mask, 'BMXHIP']

    # Create participant ID
    dashboard_data['Participant_ID'] = ['P' + str(i+1) for i in range(len(dashboard_data))]

    # Create BMI categories
    def bmi_category(bmi):
        if pd.isna(bmi):
            return 'Unknown'
        elif bmi < 18.5:
            return 'Underweight'
        elif bmi < 25:
            return 'Normal'
        elif bmi < 30:
            return 'Overweight'
        else:
            return 'Obese'

    dashboard_data['BMI_Category'] = dashboard_data['BMXBMI'].apply(bmi_category)

    # Create WHR risk categories
    def whr_risk(row):
        if pd.isna(row['WHR']):
            return 'Unknown'
        elif row['Gender'] == 'Male':
            if row['WHR'] < 0.9:
                return 'Low Risk'
            elif row['WHR'] < 1.0:
                return 'Moderate Risk'
            else:
                return 'High Risk'
        else:
            if row['WHR'] < 0.8:
                return 'Low Risk'
            elif row['WHR'] < 0.85:
                return 'Moderate Risk'
            else:
                return 'High Risk'

    dashboard_data['WHR_Risk'] = dashboard_data.apply(whr_risk, axis=1)

    # Calculate nutritional metrics if available
    mask = ~dashboard_data['DR1TKCAL'].isna()
    if mask.any():
        # Calculate protein percentage
        dashboard_data.loc[mask, 'Protein_pct'] = dashboard_data.loc[mask, 'DR1TPROT'] * 4 / dashboard_data.loc[mask, 'DR1TKCAL'] * 100

        # Calculate carb percentage
        dashboard_data.loc[mask, 'Carb_pct'] = dashboard_data.loc[mask, 'DR1TCARB'] * 4 / dashboard_data.loc[mask, 'DR1TKCAL'] * 100

        # Calculate fat percentage
        dashboard_data.loc[mask, 'Fat_pct'] = dashboard_data.loc[mask, 'DR1TTFAT'] * 9 / dashboard_data.loc[mask, 'DR1TKCAL'] * 100

        # Calculate sugar as percentage of carbs
        mask2 = (~dashboard_data['DR1TSUGR'].isna()) & (~dashboard_data['DR1TCARB'].isna())
        dashboard_data.loc[mask2, 'Sugar_carb_ratio'] = dashboard_data.loc[mask2, 'DR1TSUGR'] / dashboard_data.loc[mask2, 'DR1TCARB'] * 100

    # ----- Create interactive selectors and plots -----

    # Create participant selector
    participant_ids = dashboard_data['Participant_ID'].tolist()
    participant_select = Select(title="Select Participant:", value=participant_ids[0], options=participant_ids)

    # Filter for the first participant initially
    initial_participant = dashboard_data[dashboard_data['Participant_ID'] == participant_ids[0]].iloc[0]

    # ----- Participant Info Panel -----

    # Create table for participant info
    participant_source = ColumnDataSource(data=dict(
        metrics=['Age', 'Gender', 'BMI', 'BMI Category', 'Height', 'Weight',
                'Waist Circumference', 'Hip Circumference', 'Waist-Hip Ratio', 'WHR Risk',
                'Total Cholesterol'],
        values=[
            initial_participant['RIDAGEYR'] if not pd.isna(initial_participant['RIDAGEYR']) else 'N/A',
            initial_participant['Gender'],
            f"{initial_participant['BMXBMI']:.1f}" if not pd.isna(initial_participant['BMXBMI']) else 'N/A',
            initial_participant['BMI_Category'],
            f"{initial_participant['BMXHT']:.1f} cm" if not pd.isna(initial_participant['BMXHT']) else 'N/A',
            f"{initial_participant['BMXWT']:.1f} kg" if not pd.isna(initial_participant['BMXWT']) else 'N/A',
            f"{initial_participant['BMXWAIST']:.1f} cm" if not pd.isna(initial_participant['BMXWAIST']) else 'N/A',
            f"{initial_participant['BMXHIP']:.1f} cm" if not pd.isna(initial_participant['BMXHIP']) else 'N/A',
            f"{initial_participant['WHR']:.2f}" if not pd.isna(initial_participant.get('WHR')) else 'N/A',
            initial_participant['WHR_Risk'] if not pd.isna(initial_participant.get('WHR_Risk')) else 'N/A',
            f"{initial_participant['LBXTC']:.0f} mg/dL" if not pd.isna(initial_participant['LBXTC']) else 'N/A'
        ]
    ))

    # Create table columns
    columns = [
        TableColumn(field="metrics", title="Health Metric"),
        TableColumn(field="values", title="Value")
    ]

    # Create data table
    info_table = DataTable(source=participant_source, columns=columns,
                         width=400, height=300, index_position=None)

    # ----- Nutritional Info Panel -----

    # Create source for nutrition data
    nutrition_source = ColumnDataSource(data=dict(
        metrics=['Total Calories', 'Protein', 'Carbohydrates', 'Fiber', 'Sugar',
                'Total Fat', 'Sodium', 'Protein % of Calories', 'Carbs % of Calories',
                'Fat % of Calories', 'Sugar % of Carbs'],
        values=[
            f"{initial_participant['DR1TKCAL']:.0f} kcal" if not pd.isna(initial_participant['DR1TKCAL']) else 'N/A',
            f"{initial_participant['DR1TPROT']:.1f} g" if not pd.isna(initial_participant['DR1TPROT']) else 'N/A',
            f"{initial_participant['DR1TCARB']:.1f} g" if not pd.isna(initial_participant['DR1TCARB']) else 'N/A',
            f"{initial_participant['DR1TFIBE']:.1f} g" if not pd.isna(initial_participant['DR1TFIBE']) else 'N/A',
            f"{initial_participant['DR1TSUGR']:.1f} g" if not pd.isna(initial_participant['DR1TSUGR']) else 'N/A',
            f"{initial_participant['DR1TTFAT']:.1f} g" if not pd.isna(initial_participant['DR1TTFAT']) else 'N/A',
            f"{initial_participant['DR1TSODI']:.0f} mg" if not pd.isna(initial_participant['DR1TSODI']) else 'N/A',
            f"{initial_participant.get('Protein_pct', 'N/A'):.1f}%" if not pd.isna(initial_participant.get('Protein_pct')) else 'N/A',
            f"{initial_participant.get('Carb_pct', 'N/A'):.1f}%" if not pd.isna(initial_participant.get('Carb_pct')) else 'N/A',
            f"{initial_participant.get('Fat_pct', 'N/A'):.1f}%" if not pd.isna(initial_participant.get('Fat_pct')) else 'N/A',
            f"{initial_participant.get('Sugar_carb_ratio', 'N/A'):.1f}%" if not pd.isna(initial_participant.get('Sugar_carb_ratio')) else 'N/A'
        ]
    ))

    # Create nutrition table columns
    nutrition_columns = [
        TableColumn(field="metrics", title="Nutrient"),
        TableColumn(field="values", title="Value")
    ]

    # Create nutrition data table
    nutrition_table = DataTable(source=nutrition_source, columns=nutrition_columns,
                             width=400, height=300, index_position=None)

    # ----- JavaScript callback to update all visualizations when a new participant is selected -----

    callback = CustomJS(args=dict(
        participant_source=participant_source,
        nutrition_source=nutrition_source,
        dashboard_data=dashboard_data.to_dict('records'),
        participant_ids=participant_ids
    ), code="""
        // Get the selected participant ID
        const selected_id = cb_obj.value;

        // Find the participant data
        const participant = dashboard_data.find(p => p.Participant_ID === selected_id);

        if (participant) {
            // Update participant info table
            participant_source.data.values = [
                participant.RIDAGEYR !== null ? participant.RIDAGEYR : 'N/A',
                participant.Gender,
                participant.BMXBMI !== null ? participant.BMXBMI.toFixed(1) : 'N/A',
                participant.BMI_Category,
                participant.BMXHT !== null ? participant.BMXHT.toFixed(1) + ' cm' : 'N/A',
                participant.BMXWT !== null ? participant.BMXWT.toFixed(1) + ' kg' : 'N/A',
                participant.BMXWAIST !== null ? participant.BMXWAIST.toFixed(1) + ' cm' : 'N/A',
                participant.BMXHIP !== null ? participant.BMXHIP.toFixed(1) + ' cm' : 'N/A',
                participant.WHR !== null ? participant.WHR.toFixed(2) : 'N/A',
                participant.WHR_Risk || 'N/A',
                participant.LBXTC !== null ? participant.LBXTC.toFixed(0) + ' mg/dL' : 'N/A'
            ];

            // Update nutrition info table
            nutrition_source.data.values = [
                participant.DR1TKCAL !== null ? participant.DR1TKCAL.toFixed(0) + ' kcal' : 'N/A',
                participant.DR1TPROT !== null ? participant.DR1TPROT.toFixed(1) + ' g' : 'N/A',
                participant.DR1TCARB !== null ? participant.DR1TCARB.toFixed(1) + ' g' : 'N/A',
                participant.DR1TFIBE !== null ? participant.DR1TFIBE.toFixed(1) + ' g' : 'N/A',
                participant.DR1TSUGR !== null ? participant.DR1TSUGR.toFixed(1) + ' g' : 'N/A',
                participant.DR1TTFAT !== null ? participant.DR1TTFAT.toFixed(1) + ' g' : 'N/A',
                participant.DR1TSODI !== null ? participant.DR1TSODI.toFixed(0) + ' mg' : 'N/A',
                participant.Protein_pct !== null ? participant.Protein_pct.toFixed(1) + '%' : 'N/A',
                participant.Carb_pct !== null ? participant.Carb_pct.toFixed(1) + '%' : 'N/A',
                participant.Fat_pct !== null ? participant.Fat_pct.toFixed(1) + '%' : 'N/A',
                participant.Sugar_carb_ratio !== null ? participant.Sugar_carb_ratio.toFixed(1) + '%' : 'N/A'
            ];

            // Emit changes to update tables
            participant_source.change.emit();
            nutrition_source.change.emit();
        }
    """)

    # Attach callback to participant selector
    participant_select.js_on_change('value', callback)

    # Create dashboard layout
    dashboard = column(
        participant_select,
        row(
            column(info_table, nutrition_table)
        )
    )



    return dashboard

***Visualization 5: Health Metrics and Dietary Pattern Dashboard***

Each participant's dietary profile (calories, macronutrients, fiber, sugar, sodium, and percent breakdowns) and critical health metrics (age, gender, BMI & category, height/weight, waist & hip measures, WHR & risk, and cholesterol) are shown side by side in this interactive dashboard.

***Key Insights***
- BMI is in line with central obesity: People with a BMI of 46.0, for example, are classified as obese. They also have greater cholesterol, a larger waist circumference, and an increased risk of WHR.
- Severe nutritional imbalances: The participant with the highest BMI (46.0) consumes 107 g of sugar and 35.1% of calories from fat, whereas the person with the second-highest BMI (42.6) logs 3,849 kcal and 228 g of sugar.
- BMI and WHR risk are similar: The majority of people who are categorized as "High Risk" by WHR also had BMIs that fall into the overweight/obese range.
- Typical excess sodium: The recommended sodium intake of 2,300 mg per day is often exceeded, even by people with moderate BMI.
- Data gaps: A number of high-risk people had incomplete dietary records (designated as "NaN"), indicating areas that require better data collection.

***Implications***
- Participants with elevated cardiometabolic risk based on their anthropometric and nutritional profiles can be quickly identified with a comprehensive dashboard.
- Correlating diet patterns with health metrics uncovers potential nutritional drivers of obesity and central adiposity.
- Targeted dietary counseling is supported by the instantaneous visibility of outlier cases (such as excessive consumption of sugar or fat).
- When combined, WHR and BMI improve risk categorization beyond what can be achieved with just one measure.
- The identification of missing dietary data emphasizes how crucial comprehensive recall collection is to precise risk assessment.

In [ ]:
# Create all visualizations and organize them into tabs
def create_visualization_tabs():
    # Create each visualization
    bmi_plot = create_bmi_scatter()
    macro_plot = create_macronutrient_chart()
    whr_plot = create_waist_hip_bmi_plot()
    nutrient_analysis = create_nutrient_analysis()
    dashboard = create_health_diet_dashboard()

    # Create a simple layout with all visualizations
    from bokeh.layouts import column

    # Add header text for each visualization
    from bokeh.models import Div

    bmi_header = Div(text="<h2>BMI Distribution by Age and Gender</h2>")
    macro_header = Div(text="<h2>Macronutrient Distribution by Participant</h2>")
    whr_header = Div(text="<h2>Waist-to-Hip Ratio vs. BMI</h2>")
    nutrient_header = Div(text="<h2>Interactive Nutrient Analysis</h2>")
    dashboard_header = Div(text="<h2>Health Metrics and Dietary Pattern Dashboard</h2>")

    # Create a layout with all visualizations
    layout = column(
        bmi_header, bmi_plot,
        macro_header, macro_plot,
        whr_header, whr_plot,
        nutrient_header, nutrient_analysis,
        dashboard_header, dashboard,
        sizing_mode="stretch_width"
    )

    return layout

# Main function to run everything
def main():
    # Create visualization layout
    visualization_layout = create_visualization_tabs()

    # Output to file
    output_file("health_data_visualizations.html")

    # Save the visualizations to file
    save(visualization_layout)

    from IPython.display import HTML, display

    # assumes the notebook server is serving the current working directory
    url = "./health_data_visualizations.html"
    display(HTML(f"<a href='{url}' target='_blank'>🚀 View your dashboard</a>"))


# Run the main function
if __name__ == "__main__":
    main()

***1.3 Overall Insights from the Data Analysis***

With the majority of participants being overweight or obese by BMI and many exceeding appropriate waist-to-hip ratios, the group exhibits a significant burden of excess weight and central adiposity. The necessity of focused intervention is highlighted by a noteworthy subset that demonstrates several cardiometabolic risk markers, including elevated WHR, elevated cholesterol, and high BMI.

These hazards are exacerbated by dietary trends. Unbalances in macronutrients are common, protein is typically inadequate, and percentages of fat and carbohydrates usually fall outside of advised ranges. Fiber consumption hardly ever reaches the 25 g/day target, and added-sugar intake frequently exceeds 50 g/day (with extremes exceeding 200 g). Nutrient quality is still below ideal even when caloric intake is sufficient or excessive.

Both young and old exhibit extreme diets, and males tend toward higher WHR, while females show wider fluctuation in BMI. These hazardous trends are age- and gender-specific. Interactive dashboards that connect individual metrics and nutrient intake facilitate the quick detection of outliers (such as teenagers who consume too much sugar or elderly who consume a lot of fat) and promote individualized dietary advice aimed at reducing added sugars, increasing fiber, and rebalancing macronutrients.

In [ ]:
#-------------Logistic Regression-----------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

# 1. Load data
df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_master_imputed.csv")

# 2. Create target
df['Overweight_Obese'] = (df['BMXBMI'] >= 25).astype(int)

# 3. Compute proxy features
df['WHR'] = df['BMXWAIST'] / df['BMXHIP']

# 4. Define feature list *without* BMXBMI
features = [
    'RIDAGEYR',      # age
    'RIAGENDR',      # gender
    'BMXWAIST',      # waist circumference
    'BMXHIP',        # hip circumference
    'WHR',           # waist-to-hip ratio
    'LBXTC',         # total cholesterol
    'DR1TKCAL',      # calories
    'DR1TPROT',      # protein
    'DR1TCARB',      # carbs
    'DR1TSUGR',      # sugar
    'DR1TFIBE',      # fiber
    'DR1TTFAT',      # total fat
    'DR1TSODI'       # sodium
]

# 5. Drop rows with missing in any of the chosen features or target
data = df[features + ['Overweight_Obese']].dropna()

X = data[features]
y = data['Overweight_Obese']

# 6. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# 7. One-hot encode gender
X_train = pd.get_dummies(X_train, columns=['RIAGENDR'], drop_first=True)
X_test  = pd.get_dummies(X_test,  columns=['RIAGENDR'], drop_first=True)

# 8. Hyperparameter tuning with cross-validation
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear'],
    'class_weight': ['balanced']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    LogisticRegression(random_state=42),
    param_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1
)
grid.fit(X_train, y_train)

print(f"Best CV AUC: {grid.best_score_:.3f}")
print(f"Best params: {grid.best_params_}")

# 9. Evaluate on test set
best_lr = grid.best_estimator_
y_prob = best_lr.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.50).astype(int)

print("\n--- Test set performance (threshold = 0.50) ---")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))



Best CV AUC: 0.972
Best params: {'C': 10, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

--- Test set performance (threshold = 0.50) ---
ROC AUC: 0.973
Confusion Matrix:
 [[405  30]
 [ 99 894]]

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.93      0.86       435
           1       0.97      0.90      0.93       993

    accuracy                           0.91      1428
   macro avg       0.89      0.92      0.90      1428
weighted avg       0.92      0.91      0.91      1428



In [ ]:
#-----------Random Forest classifier----------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

# 1. Load data
df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\T1- 2025\SIT731-Data Wrangling\1.7HD\NHANES_master_imputed.csv")

# 2. Create binary target
df['Overweight_Obese'] = (df['BMXBMI'] >= 25).astype(int)

# 3. Compute WHR proxy feature
df['WHR'] = df['BMXWAIST'] / df['BMXHIP']

# 4. Define features (same as LR)
features = [
    'RIDAGEYR',      # age
    'RIAGENDR',      # gender
    'BMXWAIST',      # waist
    'BMXHIP',        # hip
    'WHR',           # waist-to-hip
    'LBXTC',         # cholesterol
    'DR1TKCAL',      # calories
    'DR1TPROT',      # protein
    'DR1TCARB',      # carbs
    'DR1TSUGR',      # sugar
    'DR1TFIBE',      # fiber
    'DR1TTFAT',      # total fat
    'DR1TSODI'       # sodium
]

# 5. Drop any rows with missing features/target
data = df[features + ['Overweight_Obese']].dropna()
X = data[features]
y = data['Overweight_Obese']

# 6. Train/test split (30%, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# 7. One-hot encode gender exactly as before
X_train = pd.get_dummies(X_train, columns=['RIAGENDR'], drop_first=True)
X_test  = pd.get_dummies(X_test,  columns=['RIAGENDR'], drop_first=True)

# 8. Set up CV + grid search for RandomForest
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [5, 10, 15, None],
    'class_weight': ['balanced']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1
)
grid_rf.fit(X_train, y_train)

print(f"Best CV AUC: {grid_rf.best_score_:.3f}")
print(f"Best params: {grid_rf.best_params_}")

# 9. Evaluate on the test set
best_rf = grid_rf.best_estimator_
y_prob_rf = best_rf.predict_proba(X_test)[:,1]
y_pred_rf = (y_prob_rf >= 0.5).astype(int)

print("\n--- Test set performance (threshold = 0.50) ---")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_rf):.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))


Best CV AUC: 0.969
Best params: {'class_weight': 'balanced', 'max_depth': 15, 'n_estimators': 300}

--- Test set performance (threshold = 0.50) ---
ROC AUC: 0.972
Confusion Matrix:
 [[371  64]
 [ 53 940]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.85      0.86       435
           1       0.94      0.95      0.94       993

    accuracy                           0.92      1428
   macro avg       0.91      0.90      0.90      1428
weighted avg       0.92      0.92      0.92      1428



***1.4 Model Comparison: Logistic Regression vs. Random Forest***

We trained two different classifiers to predict whether a participant is overweight/obese (BMI ≥ 25) using the same features, train/test split, and evaluation threshold (0.50). Here’s how they stack up:
| Metric                  | Logistic Regression | Random Forest |
| ----------------------- | ------------------- | ------------- |
| **Cross-Val AUC**       | 0.972               | 0.969         |
| **Test-Set AUC**        | 0.973               | 0.972         |
| **Accuracy**            | 0.91                | 0.92          |
| **Precision (Class 1)** | 0.97                | 0.94          |
| **Recall (Class 1)**    | 0.90                | 0.95          |
| **F1-Score (Class 1)**  | 0.93                | 0.94          |

- Discrimination (AUC): Both models have very high AUCs (> 0.97), which shows that they are highly good at classifying participants according to risk. The difference between 0.973 and 0.972 is insignificant, yet logistic regression wins out by a small margin.
- Overall Accuracy: Overall accuracy is slightly higher for Random Forest (92% vs. 91%), but both are good given class balance.
- Class-Specific Performance:
   - Precision (avoiding false positives): With a higher logistic regression coefficient (0.97 vs. 0.94), its "overweight/obese" predictions are more frequently accurate.
   - Recall (capturing true positives): Because random forest performs better (0.95 vs. 0.90), it misses fewer real cases of overweight or obesity.
   - This trade-off is reflected in the extremely similar F1-scores (harmonic mean) (0.93 vs. 0.94).

***Which to Choose?***
- The greater precision of logistic regression might be better if reducing false alarms—that is, just identifying people who are most likely overweight or obese—is ones top concern.
- The higher recall of random forest is useful if catching as many true cases as possible is required, even if it means a few more false positives.
- The simpler logistic regression may be adequate for most applications due to the near-tie in AUC and overall performance; however, the random forest is a good substitute if one expects complicated non-linear interactions or wants the little increase in recall.

***1.5 Ethical Considerations and Data Privacy Issues***

***1. Privacy and Re-identification Risk***
- Concern: Particularly in a small sample, re-identification can be made possible even with pseudonymized participant IDs when paired with comprehensive health and dietary indicators (age, BMI, waist/hip measurements, nutrient intakes).
- Mitigation: In public outputs, avoid sharing unique combinations of features, use broader categories (e.g., age bands instead of specific ages), and mask extreme values with modest random noise.

***2. Data Security***
- Concern: Sensitive health information may be exposed by unauthorized access or scraping of interactive dashboards and the underlying CSV files.
- Mitigation: Implement stringent access controls, host visualizations behind authenticated portals, encrypt data in transit and at rest (HTTPS, encrypted storage), and regularly audit all servers and data handling programs for security flaws.

***3. Stigmatization and Bias***
- Concern: People might be shamed and subjected to harmful stereotypes when they are labeled as "obese" or "high risk." Results may be overgeneralized by readers due to small, non-representative samples.
- Mitigation: Employ neutral, clinical terminology (e.g. “BMI ≥30” rather than “obese”), clearly state sample size limitations, and use color palettes and legends that avoid alarmist connotations.

***4. Informed Consent and Scope of Use***
- Concern: It's possible that participants didn't expect their de-identified data to be integrated across several domains or feed into interactive dashboards that are accessible to the public.
- Mitigation: Future research should make sure that participants receive private previews of how their data will appear, offer opt-out alternatives for specific analyses, and contain explicit consent language outlining possible data visualization and aggregation.

***5. Interpretation and Causal Overreach***
- Concern: Correlations shown (e.g. high sugar intake alongside high BMI) may be misconstrued as causal, and small sample size can produce spurious associations.
- Mitigation: Display sample sizes and confidence intervals when appropriate, prominently state that "correlation does not imply causation," and clearly state that the analysis is exploratory for each visualization.

***6. Algorithmic and Metric Bias***
- Concern: Standard BMI, WHR, and nutritional reference standards might not take age, sex, or ethnic variations into consideration, which could lead to some people being incorrectly classified.
- Mitigation: When possible, include alternative or age-adjusted metrics, acknowledge the demographic specificity of cut-offs in documentation, and consult subject-matter experts to validate any threshold values that are employed.

***1.6 Conclusion***

Our research clearly demonstrated higher cardiometabolic risk across the board, utilizing a comprehensive, integrated NHANES dataset that incorporated anthropometric, biochemical, nutritional, and behavioral measurements. Interactive scatterplots and stacked-bar charts showed widespread patterns of central adiposity (high WHR), overweight and obesity, and dietary imbalances, including too much sodium and added sugars and not enough fiber. These results were made tangible at the individual level by the participant dashboard, which made it possible to quickly identify outliers for focused intervention.

Based on these descriptive findings, we created and refined two prediction models to differentiate between those who are normal weight and those who are overweight or obese: logistic regression and a random forest classifier. On the held-out test set, both methods demonstrated high discrimination (ROC AUC = 0.97) and comparable accuracy (~ 0.91–0.92), indicating that the risk signal is robustly captured by essential parameters (age, gender, waist and hip measures, cholesterol, and specific food intakes). Although both models performed fairly well, the random forest model marginally beat logistic regression in recall for the overweight/obese class, suggesting its benefit in capturing nonlinear relationships.

Predictive modeling and visualizations work together to highlight the necessity of individualized preventative strategies: interactive dashboards inform lifestyle advice, while data-driven risk scores can identify high-risk individuals. Our insights and predictive capacity will be further refined in the future by increasing the sample size, adding longitudinal follow-up, and integrating more detailed dietary and genetic data—always supported by strict consent, privacy protections, and open governance.